In [ ]:
import pandas as pd
import numpy as np 
import cv2
import matplotlib.pyplot as plt
from scipy import ndimage
from skimage.color import rgb2gray
import itertools
import os
from sklearn.model_selection import train_test_split
import time
from sklearn import preprocessing
from sklearn import metrics
import tifffile

In [ ]:
#test-train split
imageData = os.listdir("./data/images")
imageData = [imd for imd in imageData if imd.split('.')[-1] == 'jpg']
imageLabel = os.listdir("./data/labels")
imageLabel = [iml for iml in imageLabel if iml.split('.')[-1] == 'jpg']
#ensure image and labels correspond
imageData.sort()
imageLabel.sort()
trainImage,validImage,trainLable,validLabel = train_test_split(imageData,imageLabel,test_size=0.2,shuffle=True,random_state=2)
def addPrefix(x):
    pre = x.split("-")[1][0:3]
    if(pre == "vol"):
        return "./data/images/" + x
    elif(pre == "lab"):
        return "./data/labels/" + x
    else:
        return "Invalid input"
tmp = [trainImage,validImage,trainLable,validLabel] 

trainImage = list(map(addPrefix,trainImage))
validImage = list(map(addPrefix,validImage))
trainLable = list(map(addPrefix,trainLable))
validLabel = list(map(addPrefix,validLabel))

finalImage = trainImage+validImage

In [ ]:
def preprocess(image):
    iv = image
    ret,img = cv2.threshold(iv,125,255,cv2.THRESH_BINARY)
    _,contours,hierarchy = cv2.findContours(img,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    thres_blob_area = 20
    for i in range(len(contours)):
        index_level = int(hierarchy[0][i][1])
        if index_level<= i:
            cnt = contours[i]
            area = cv2.contourArea(cnt)
            if area <= thres_blob_area:
                cv2.drawContours(img,[cnt],-1,255,-1,1)
    return img

In [ ]:
def patchDiv(list1,labelPath,patchSize=256):
    #loop through image and label

    imagePatch = []
    labelPatch = []
    for i in list1:
        if(i[-3:] == "jpg"):
            imageNumber = i.split(".jpg")[0][-2:]
            imageLabel = "/train-labels" + imageNumber + ".jpg"
            ig = cv2.imread(i,0)
            img = preprocess(ig)
            label = cv2.imread(labelPath + imageLabel,0)
            
            cutW = img.shape[0]//patchSize
            cutH = img.shape[1]//patchSize
            #divide image into patch
            for w in range(cutW):
                for h in range(cutH):
                    imagePatch.append(img[w*patchSize:(w+1)*patchSize, h*patchSize:(h+1)*patchSize])
                    labelPatch.append(label[w*patchSize:(w+1)*patchSize, h*patchSize:(h+1)*patchSize])
    return imagePatch,labelPatch

In [ ]:
# implement image patching 
imgP,labeP = patchDiv(trainImage,"./data/labels",patchSize=256)
print("number of image patch",len(imgP))
print("number of label patch",len(labeP))
print(imgP[0].shape)

In [ ]:
def fact(mean,h,n):

    return fact_iter(0,0,mean,h,n)

def fact_iter(product, count, mean,h,n):

    if count >= len(n):

        return product

    return fact_iter(product+(n[count]-mean)**h, count+1,mean,h,n)

In [ ]:
pick = [[0,0],[0,1],[0,2],[1,0],[1,1],[1,2],[2,0],[2,1],[2,2]]

In [ ]:
import pandas as pd
import numpy as np 
import cv2
def patchGen(img,label,patch_size,npatch):
    h,w = img.shape[0:2]
    #patch_size should be odd
    if(patch_size%2 == 0):
        patch_size += 1
    #image side cut
    cut = patch_size // 2
    #randomly select npatch patches
    patchMatrx = []
    patchLabel = []
    for i in range(npatch):
        randomHeight = np.random.randint(low = cut,high = h-cut,size = 1)
        randomWeight = np.random.randint(low = cut,high = w-cut,size = 1)
        labelPixel = label[randomWeight,randomHeight]
        rowS = randomWeight-cut
        colS = randomHeight-cut
        matrxImg = []

        for j in range(patch_size):#0,1,2
            rowImg = []
 
            for k in range(patch_size):
                pixel = img[rowS+k,colS+j]
                rowImg.append(pixel)
            matrxImg.append(rowImg)
       
        patchMatrx.append(matrxImg)
        patchLabel.append(labelPixel)
    return np.array(patchMatrx),np.array(patchLabel)  

In [ ]:
def T_gener(numpy):
    training_para = []
    for i in numpy:
        T = np.zeros((1,24))
        for j in range(len(pick)):
            T[0][j] = i[pick[j][0],pick[j][1]]
        T[0][9] = np.median(i)
        T[0][10] = np.max(i)-np.min(i)
        T[0][11] = fact(0,2,T[0][:9])
        T[0][12] = fact(T[0][9],2,T[0][:9])/9
        T[0][13] = fact(T[0][9],3,T[0][:9])/9
        T[0][14] = fact(T[0][9],4,T[0][:9])/9
        G = np.gradient(i)[0]
        for k in range(len(pick)):
            T[0][15+k] = G[pick[k][0],pick[k][1]]
        training_para.append(T)
    return np.array(training_para)

In [ ]:
def train_model(X, y):
    start = time.time()
    from sklearn.svm import SVC
    print ('[INFO] Training Support Vector Machine model.')
    model = SVC(gamma = "auto",kernel = 'rbf',C = 1000)
    model.fit(X, y)

    print ('[INFO] Model training complete.')
    print ('[INFO] Training Accuracy: %.2f' %model.score(X, y))
    stop = time.time()
    d = stop - start
    print('Running duration is {}'.format(d))
    return model

In [ ]:
def final_generator(list1,list2):
    for_train = []
    for_test = []
    for path in range(len(list1)):
        img = list1[path]
        ret1,lab1 = cv2.threshold(list2[path],127,255,cv2.THRESH_BINARY)
        patch_size = 3
        patches,label_patch = patchGen(img,lab1,3,1000)
        patchNeed = patches.reshape((patches.shape[0],patches.shape[1],patches.shape[2]))
        Train = T_gener(patchNeed)
        aft = Train.reshape((Train.shape[0],-1))
        aft = preprocessing.maxabs_scale(aft,axis=0, copy=True)
        for_train.append(aft)
        for_test.append(label_patch)
        affor_train = np.array(for_train).reshape((-1,24))
        affor_test = np.array(for_test).reshape((-1,1))
    return affor_train,affor_test

In [ ]:
def pre(X,model):
    pred = model.predict(X)
    return pred

In [ ]:
def evalPatch(testImg,patch_size):
    h,w = testImg.shape[0:2]
    #patch_size should be odd
    if(patch_size%2 == 0):
        patch_size += 1
    #image side cut
    cut = patch_size // 2  
    patches = []
    for row in range(cut,w-cut):
        for col in range(cut,h-cut):
            rowS = row-cut
            colS = col-cut
            matrxImg = []
            for j in range(patch_size):#0,1,2
                rowImg = []
                for k in range(patch_size):
                    pixel = testImg[rowS+k,colS+j]
                    rowImg.append(pixel)
                matrxImg.append(rowImg)
            patches.append(matrxImg)
    return np.array(patches)

In [ ]:
def mergeResult(imageList,originalImageShape=512,patchSize=256):
    #index: number of elements in the input list for one image
    cut = originalImageShape//patchSize
    index = cut * cut
    count = 0
    i = 0
    mergedResult = np.zeros((len(imageList)//index, ) + (originalImageShape,originalImageShape))
    while(count + index <= len(imageList)):#(4,8,12...) <= 4
        vstacked = []
        temp = imageList[count:count + index]#[0:4]
        flag = 0
        for h in range(cut,index):#(2-4)
            for r in range(flag,flag + cut):#(0-2),(2-4)
                hstacked = temp[r] if(r == flag) else np.hstack((hstacked,temp[r]))
            vstacked = hstacked if (len(vstacked)==0) else np.vstack((vstacked,hstacked))
            flag = h      
        count += index
        mergedResult[i] = vstacked
        i += 1 
    return mergedResult

In [ ]:
def postprocess(list_of_pred):
    final_image = []
    for pro_c in list_of_pred:
        img_blur_G_first = cv2.GaussianBlur(pro_c,(5,5),0)
        ret1,th1 = cv2.threshold(img_blur_G_first,160,255,cv2.THRESH_BINARY)
        img_blur_G_second = cv2.GaussianBlur(th1,(5,5),0)
        ree,tth =  cv2.threshold(img_blur_G_second,160,255,cv2.THRESH_BINARY)
        fimg = tth/255
        final_image.append(fimg)
    return final_image

In [ ]:
def saveResult(save_path,npyfile,flag_multi_class = False,num_class = 2):
    stacked = []
    for i,item in enumerate(npyfile):
        img = item[:,:].astype('float32')
        img = np.expand_dims(img,axis=0)
        if(len(stacked)==0):
            stacked = img
        else:
            stacked = np.vstack((stacked,img))
    tifffile.imsave(save_path + '/allPredict.tif', stacked)

In [ ]:
TTrain,Tlabel = final_generator(imgP,labeP)
Tlabel = Tlabel.ravel()
model = train_model(TTrain,Tlabel)

In [ ]:
valP,valmP = patchDiv(finalImage,"./data/labels",patchSize=256)

In [ ]:
test_d = []
for lm in valP:
    img = lm
    ee = evalPatch(img,3)
    test_d.append(ee)

In [ ]:
test_v = []
for td in test_d:
    tee = T_gener(td)
    aft = tee.reshape((tee.shape[0],-1))
    aft = preprocessing.maxabs_scale(aft,axis=0, copy=True)
    test_v.append(aft)

In [ ]:
#creat list of prediction outputs
test_pre = []
for vv in test_v:
    pret =  pre(vv,model)
    pret = pret.reshape((254,254))
    padimg= np.pad(pret, ((1, 1), (1, 1)), 'constant', constant_values=(0, 0))
    test_pre.append(padimg)

In [ ]:
mergedResult = mergeResult(test_pre,originalImageShape=512,patchSize=256)
npyfile = postprocess(mergedResult)
saveResult('./',npyfile,flag_multi_class = False,num_class = 2)